# 前処理

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [2]:
from replace import Country
country = Country()

pd.options.display.float_format = '{:.4f}'.format

visitors = pd.read_csv('data/visitors.csv')
gdp = pd.read_csv('data/gdp.csv')
currency = pd.read_csv('data/currency.csv')
continent = pd.read_csv('data/continent.csv')
cpi = pd.read_csv('data/cpi_japan.csv')
distance = pd.read_csv('data/distance.csv')
population = pd.read_csv('data/population.csv')

In [3]:
visitors["Country"] = visitors["Country"].replace(country.replace_list)
visitors = visitors.loc[~visitors["Country"].isin(country.list)]
visitors = visitors.loc[visitors["Year"] < 2020]

gdp = gdp.melt(id_vars="Country", var_name="Year", value_name="GDP")
gdp["Year"] = gdp.Year.astype(int)

population = population.melt(id_vars="Country", var_name="Year", value_name="Population")
population["Year"] = population.Year.astype(int)

In [4]:
df = pd.merge(visitors, currency, on=["Country", "Year"], how="inner")
df = pd.merge(df, gdp, on=["Country", "Year"], how="inner")
df = pd.merge(df, cpi, on=["Country", "Year"], how="inner")
df = pd.merge(df, population, on=["Country", "Year"], how="inner")

In [5]:
cluster1 = [
        'Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland',
        'France', 'Germany', 'Ireland', 'Israel', 'Italy', 'Netherlands',
        'New Zealand', 'Norway', 'Singapore', 'Sweden', 'Switzerland',
        'United Kingdom', 'United States'
        ]

cluster2 = [
        'China', 'Malaysia', 'Mexico', 'Philippines', 'Poland', 'Portugal',
        'Spain', 'Thailand'
        ]

cluster3 = ['Brazil']

cluster4 = ['India', 'Indonesia', 'Russia', 'Turkey', 'Vietnam']

cluster5 = ['Hong Kong', 'Korea', 'Taiwan']

In [6]:
df["Cluster"] = 0
df.loc[df["Country"].isin(cluster1), "Cluster"] = 1
df.loc[df["Country"].isin(cluster2), "Cluster"] = 2
df.loc[df["Country"].isin(cluster3), "Cluster"] = 3
df.loc[df["Country"].isin(cluster4), "Cluster"] = 4
df.loc[df["Country"].isin(cluster5), "Cluster"] = 5

In [7]:
df["RER"] = df["Exchange Rate"] * df["CPI_Japan"] / df["CPI"]
df["GDPpc"] = df["GDP"] / df["Population"]
df["log_Visitors"] = np.log(df["Visitors"])
df["log_GDP"] = np.log(df["GDP"])
df["log_GDPpc"] = np.log(df["GDPpc"])
df["log_RER"] = np.log(df["RER"])

In [8]:
df = pd.concat([df, pd.get_dummies(df["Year"], prefix="yr", prefix_sep="", drop_first=True, dtype=int)], axis=1)

In [9]:
select = [
        'Year', 'Country', 'Cluster',
        'log_Visitors', 'log_GDP', 'log_GDPpc', 'log_RER',
        'yr2011', 'yr2012', 'yr2013',
        'yr2014', 'yr2015', 'yr2016',
        'yr2017', 'yr2018', 'yr2019'
]

df = df[select]

In [10]:
df_cluster1 = df.copy()
df_cluster2 = df.copy()
df_cluster3 = df.copy()
df_cluster4 = df.copy()
df_cluster5 = df.copy()

In [11]:
df_cluster1 = df_cluster1.loc[df["Cluster"]==1]
df_cluster2 = df_cluster2.loc[df["Cluster"]==2]
df_cluster3 = df_cluster3.loc[df["Cluster"]==3]
df_cluster4 = df_cluster4.loc[df["Cluster"]==4]
df_cluster5 = df_cluster5.loc[df["Cluster"]==5]

# PanelOLS

## cluster1

In [12]:
df_cluster1["Year"] = pd.to_datetime(df_cluster1["Year"], format="%Y")
df_cluster1.set_index(["Country", "Year"], inplace=True)

In [13]:
model = PanelOLS(df_cluster1["log_Visitors"], df_cluster1[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        0.3886     0.1511     2.5715     0.0110      0.0902      0.6870
log_RER       -0.4234     0.1579    -2.6812     0.0081     -0.7353     -0.1115
yr2011        -0.4048     0.0302    -13.400     0.0000     -0.4645     -0.3452
yr2012        -0.0968     0.0308    -3.1427     0.0020     -0.1577     -0.0360
yr2013        -0.0431     0.0434    -0.9925     0.3224     -0.1289      0.0427
yr2014         0.0792     0.0501     1.5824     0.1155     -0.0197      0.1781
yr2015         0.2672     0.0484     5.5200     0.0000      0.1716      0.3629
yr2016         0.4450     0.0389     11.452     0.0000      0.3683      0.5218
yr2017         0.4868     0.0453     10.751     0.0000      0.3974      0.5763
yr2018         0.5785     0.0482     12.010     0.00

## cluster2

In [14]:
df_cluster2["Year"] = pd.to_datetime(df_cluster2["Year"], format="%Y")
df_cluster2.set_index(["Country", "Year"], inplace=True)

In [15]:
model = PanelOLS(df_cluster2["log_Visitors"], df_cluster2[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        1.2322     0.2835     4.3460     0.0001      0.6653      1.7991
log_RER       -0.7496     0.3850    -1.9469     0.0562     -1.5195      0.0203
yr2011        -0.4265     0.0657    -6.4935     0.0000     -0.5579     -0.2952
yr2012        -0.0629     0.0678    -0.9273     0.3574     -0.1985      0.0727
yr2013         0.0008     0.0977     0.0084     0.9933     -0.1946      0.1962
yr2014         0.2702     0.1091     2.4773     0.0160      0.0521      0.4883
yr2015         0.5225     0.1096     4.7670     0.0000      0.3033      0.7417
yr2016         0.7872     0.0816     9.6498     0.0000      0.6241      0.9503
yr2017         0.8169     0.0912     8.9551     0.0000      0.6345      0.9993
yr2018         0.8795     0.0983     8.9432     0.00

## cluster4

In [18]:
df_cluster4["Year"] = pd.to_datetime(df_cluster4["Year"], format="%Y")
df_cluster4.set_index(["Country", "Year"], inplace=True)

In [19]:
model = PanelOLS(df_cluster4["log_Visitors"], df_cluster4[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        0.5147     0.6149     0.8370     0.4084     -0.7349      1.7643
log_RER       -1.4705     0.3454    -4.2577     0.0002     -2.1724     -0.7686
yr2011        -0.2330     0.1397    -1.6683     0.1044     -0.5168      0.0508
yr2012         0.0883     0.1516     0.5822     0.5643     -0.2198      0.3963
yr2013        -0.0176     0.1580    -0.1116     0.9118     -0.3388      0.3035
yr2014         0.1162     0.1729     0.6719     0.5062     -0.2352      0.4676
yr2015         0.2281     0.1889     1.2075     0.2356     -0.1558      0.6120
yr2016         0.5123     0.2161     2.3711     0.0235      0.0732      0.9514
yr2017         0.5869     0.2357     2.4898     0.0178      0.1079      1.0659
yr2018         0.8040     0.2674     3.0068     0.00

## cluster5

In [16]:
df_cluster5["Year"] = pd.to_datetime(df_cluster5["Year"], format="%Y")
df_cluster5.set_index(["Country", "Year"], inplace=True)

In [17]:
model = PanelOLS(df_cluster5["log_Visitors"], df_cluster5[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        1.1138     3.2081     0.3472     0.7330     -5.6871      7.9146
log_RER       -1.7082     1.3725    -1.2446     0.2312     -4.6178      1.2014
yr2011        -0.3190     0.1399    -2.2798     0.0367     -0.6157     -0.0224
yr2012        -0.0966     0.2143    -0.4507     0.6582     -0.5509      0.3577
yr2013        -0.1776     0.5879    -0.3021     0.7665     -1.4238      1.0686
yr2014        -0.1674     0.8176    -0.2048     0.8403     -1.9006      1.5658
yr2015         0.0010     1.0499     0.0009     0.9993     -2.2248      2.2267
yr2016         0.3372     0.9928     0.3396     0.7385     -1.7674      2.4417
yr2017         0.3930     1.1774     0.3338     0.7429     -2.1030      2.8890
yr2018         0.3914     1.2567     0.3115     0.75

# OLS (cluster3)

In [20]:
X = sm.add_constant(df_cluster3[['log_GDP', 'log_RER']])
y = df_cluster3['log_Visitors']

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           log_Visitors   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     2.014
Date:                Thu, 21 Nov 2024   Prob (F-statistic):              0.204
Time:                        14:14:50   Log-Likelihood:                0.39051
No. Observations:                  10   AIC:                             5.219
Df Residuals:                       7   BIC:                             6.127
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -117.8027     96.395     -1.222      0.2

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)
